In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image
import os

import torch
import torch.nn as nn
import torchvision.datasets as datasets
# from torchvision.datasets import MNIST
from keras.datasets import mnist
from torch.utils.data import DataLoader
import torch.optim as optim

from torchvision import transforms
import torchvision
import torchsummary

from networks.discriminator import PatchDiscriminator

Using TensorFlow backend.
/home/danjiii/Program/ML_practice/data-venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/danjiii/Program/ML_practice/data-venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/danjiii/Program/ML_practice/data-venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
class catvsdog_dataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.path = path
        self.list_photo = list(os.listdir(path))
        self.simple = transforms.Compose([
        transforms.CenterCrop(100), # converts to [0,1] interval
        transforms.ToTensor(),
        ])
        
    def __getitem__(self, idx):
        img = Image.open(f'{self.path}/{self.list_photo[idx]}')
        label_name = self.list_photo[idx].split('.')[0]
        label = 1. if label_name == 'cat' else 0.
        img = self.simple(img)
        if img.shape[1] < 100 or img.shape[2] < 100:
            self.__getitem__(idx+1)
        return (img.unsqueeze(0), 
                torch.tensor(label).type(torch.float))
    
    def __len__(self):
        return len(self.list_photo)

In [4]:
m = catvsdog_dataset('train')

In [5]:
m[150][0].shape

torch.Size([1, 3, 100, 100])

In [12]:
class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
#         if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            
#         else:
#             use_bias = norm_layer == nn.InstanceNorm2d
        use_bias = False
        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)


In [10]:
import torch.nn as nn
import torch.nn.functional as F

# helper conv function
def conv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                           kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
    
    layers.append(conv_layer)

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)

class Discriminator(nn.Module):
    
    def __init__(self, conv_dim=64):
        super(Discriminator, self).__init__()

        # Define all convolutional layers
        # Should accept an RGB image as input and output a single value

        # Convolutional layers, increasing in depth
        # first layer has *no* batchnorm
        self.conv1 = conv(3, conv_dim, 4, batch_norm=False) # x, y = 64, depth 64
        self.conv2 = conv(conv_dim, conv_dim*2, 4) # (32, 32, 128)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4) # (16, 16, 256)
        self.conv4 = conv(conv_dim*4, conv_dim*8, 4) # (8, 8, 512)
        
        # Classification layer
        self.conv5 = conv(conv_dim*8, 1, 19, stride=1, batch_norm=False)

    def forward(self, x):
        # relu applied to all conv layers but last
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        # last, classification layer
        out = self.conv5(out)
        return out

In [14]:
# model = PatchDiscriminator(c_dim=1, use_sigmoid=False, n_layers=3)
model = NLayerDiscriminator(3)
# new_model = nn.Sequential(model, )
torchsummary.summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           3,136
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         131,072
       BatchNorm2d-4          [-1, 128, 32, 32]             256
         LeakyReLU-5          [-1, 128, 32, 32]               0
            Conv2d-6          [-1, 256, 16, 16]         524,288
       BatchNorm2d-7          [-1, 256, 16, 16]             512
         LeakyReLU-8          [-1, 256, 16, 16]               0
            Conv2d-9          [-1, 512, 15, 15]       2,097,152
      BatchNorm2d-10          [-1, 512, 15, 15]           1,024
        LeakyReLU-11          [-1, 512, 15, 15]               0
           Conv2d-12            [-1, 1, 14, 14]           8,193
Total params: 2,765,633
Trainable params: 2,765,633
Non-trainable params: 0
---------------------------

In [6]:
BATCH_SIZE = 5
criterian = nn.MSELoss()
d = Mnistdenoizing_dataset(x_train, y_train)
def init_weight(layer):
    if type(layer) == nn.Conv2d or type(layer) == nn.ConvTranspose2d:
        torch.nn.init.uniform_(layer.weight)
model.apply(init_weight)
def train(model, num_epoch=10, gd=None):
    model.double()
    if gd is None:
        gd = optim.Adam(model.parameters())
    dataloader = DataLoader(d, batch_size=BATCH_SIZE, shuffle=True)
    losses = []
    for epoch in tqdm(range(num_epoch)):
        model.train(True)
        for i, batch in enumerate(dataloader):
            gd.zero_grad()
            f = model(batch[0]) # 
            loss = criterian(batch[1][0], f[0][0][0])
            loss.backward()
            losses.append(loss.data.cpu().numpy())
            gd.step()
            gd.zero_grad()
        train_mse = np.mean(losses[-(i+1):])
        
        model.train(False)
        print(f"{epoch+1}, Train loss: {train_mse}")

NameError: name 'Mnistdenoizing_dataset' is not defined

In [ ]:
# d[0][1]
train(model)
# torch.unsqueeze
# m = Mnistdenoizing_dataset(x_train, y_train)
# d = DataLoader(m, batch_size=2)


  0%|          | 0/10 [00:00<?, ?it/s]